<a href="https://colab.research.google.com/github/PremChand-Neela/Elderly-fall-detection/blob/main/finalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


WINDOW_SIZE = 50
SENSOR_COLS = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
RANDOM_STATE = 42
EPOCHS = 50
BATCH_SIZE = 64

print("Downloading dataset from Kaggle...")
dataset_path = kagglehub.dataset_download("ziya07/elderly-fall-detection-iot-dataset")

print("Dataset downloaded at:", dataset_path)


csv_files = [f for f in os.listdir(dataset_path) if f.endswith(".csv")]

if not csv_files:
    raise FileNotFoundError("No CSV files found inside downloaded Kaggle dataset folder!")

csv_path = os.path.join(dataset_path, csv_files[0])
print("Using data file:", csv_path)

df = pd.read_csv(csv_path)
df = df.dropna().reset_index(drop=True)


for c in SENSOR_COLS:
    if c not in df.columns:
        raise ValueError(f"Column '{c}' missing. Check dataset column names.")

if "label" not in df.columns:
    raise ValueError("Dataset must contain a label column named 'label'.")

X_all = df[SENSOR_COLS].values.astype(np.float32)
y_raw = df["label"].values


le = LabelEncoder()
y_int = le.fit_transform(y_raw)
num_classes = len(le.classes_)

print("Detected Classes:", list(le.classes_))


def create_windows(X, y, win):
    X_seq, y_seq = [], []
    for i in range(len(X) - win):
        X_seq.append(X[i:i+win])
        y_seq.append(y[i+win-1])  # label at end of window
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq_int = create_windows(X_all, y_int, WINDOW_SIZE)
y_seq_cat = to_categorical(y_seq_int, num_classes)

print("Window Data Shape:", X_seq.shape)


X_train, X_test, y_train_cat, y_test_cat, y_train_int, y_test_int = train_test_split(
    X_seq, y_seq_cat, y_seq_int, test_size=0.2, random_state=RANDOM_STATE, stratify=y_seq_int
)


timesteps = X_train.shape[1]
features = X_train.shape[2]

model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(timesteps, features)),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.3),

    LSTM(100),
    Dropout(0.4),

    Dense(50, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop]
)


loss, acc = model.evaluate(X_test, y_test_cat)
print("\nTest Accuracy:", acc)

y_pred = np.argmax(model.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test_int, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test_int, y_pred))


def predict_fall(window):
    if window.shape != (WINDOW_SIZE, len(SENSOR_COLS)):
        raise ValueError(f"Window shape must be ({WINDOW_SIZE}, {len(SENSOR_COLS)})")

    inp = np.expand_dims(window, axis=0)
    pred = model.predict(inp)[0]
    cls = np.argmax(pred)
    label = le.inverse_transform([cls])[0]

    if "fall" in label.lower() or "fallen" in label.lower():
        print("FALL DETECTED")
    else:
        print("NOT FALLEN")

    return label

print("\nTesting prediction on a random window...")

sample = X_test[0]
true_label = le.inverse_transform([y_test_int[0]])[0]
pred_label = predict_fall(sample)

print("True Label:", true_label)
print("Predicted Label:", pred_label)


100%|██████████| 3.53G/3.53G [00:39<00:00, 95.2MB/s]

Extracting files...


Dataset downloaded at: /root/.cache/kagglehub/datasets/ziya07/elderly-fall-detection-iot-dataset/versions/2
Using data file: /root/.cache/kagglehub/datasets/ziya07/elderly-fall-detection-iot-dataset/versions/2/fall_detection.csv
Detected Classes: ['bend', 'fall_backward', 'fall_forward', 'fall_side_left', 'fall_side_right', 'fall_slump', 'lie_down', 'sit', 'stand', 'walk']
Window Data Shape: (24950, 50, 6)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 48, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,032 (285.28 KB)

 Trainable params: 72,904 (284.78 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 57ms/step - accuracy: 0.2240 - loss: 2.0679 - val_accuracy: 0.2290 - val_loss: 2.0181
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - accuracy: 0.2976 - loss: 1.8360 - val_accuracy: 0.3439 - val_loss: 1.7634
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.3680 - loss: 1.6950 - val_accuracy: 0.4907 - val_loss: 1.4664
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - accuracy: 0.4744 - loss: 1.4783 - val_accuracy: 0.6240 - val_loss: 1.1051
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.5820 - loss: 1.2080 - val_accuracy: 0.7472 - val_loss: 0.8049
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - accuracy: 0.6735 - loss: 0.9682 - val_accuracy: 0.7951 - val_loss: 0.6284
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 54ms/step - accuracy: 0.7402 - loss: 0.7965 - val_accuracy: 0.8587 - val_loss: 0.4672
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 54ms/step - accuracy: 0.7828 - loss: 0.6470 - 

In [2]:
print("\nTesting prediction on a random window...")

sample = X_test[0]
true_label = le.inverse_transform([y_test_int[0]])[0]
pred_label = predict_fall(sample)

print("True Label:", true_label)
print("Predicted Label:", pred_label)


Testing prediction on a random window...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
NOT FALLEN
True Label: stand
Predicted Label: stand


In [3]:
print("\nTesting prediction on a random window...")

sample = X_test[0]
true_label = le.inverse_transform([y_test_int[0]])[0]
pred_label = predict_fall(sample)

print("True Label:", true_label)
print("Predicted Label:", pred_label)


Testing prediction on a random window...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
NOT FALLEN
True Label: stand
Predicted Label: stand


In [4]:
print("\nTesting prediction on a new random window...")

sample_new = X_test[1] # Using a different sample from X_test
true_label_new = le.inverse_transform([y_test_int[1]])[0]
pred_label_new = predict_fall(sample_new)

print("True Label (new sample):", true_label_new)
print("Predicted Label (new sample):", pred_label_new)


Testing prediction on a new random window...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
NOT FALLEN
True Label (new sample): sit
Predicted Label (new sample): sit


In [5]:
fall_labels = [i for i, label in enumerate(le.classes_) if 'fall' in label.lower()]
print("Indices of 'fall' classes:", fall_labels)

# Filter X_test and y_test_int for fall-related classes
fall_indices = np.isin(y_test_int, fall_labels)
X_fall_test = X_test[fall_indices]
y_fall_test_int = y_test_int[fall_indices]
y_fall_test_cat = y_test_cat[fall_indices]

print(f"Shape of X_fall_test: {X_fall_test.shape}")
print(f"Shape of y_fall_test_int: {y_fall_test_int.shape}")

Indices of 'fall' classes: [1, 2, 3, 4, 5]
Shape of X_fall_test: (2340, 50, 6)
Shape of y_fall_test_int: (2340,)


Now that we have isolated the fall-related test data, we can evaluate the `predict_fall` function specifically on these samples. This will show us how well the model performs when it *should* be detecting a fall.

In [6]:
print("\nEvaluating predict_fall on fall-only test data...")

y_pred_fall = np.argmax(model.predict(X_fall_test), axis=1)

print("Classification Report for Fall Classes:", le.classes_[fall_labels])
print(classification_report(y_fall_test_int, y_pred_fall, target_names=le.classes_))
print("Confusion Matrix for Fall Classes:\n", confusion_matrix(y_fall_test_int, y_pred_fall))


Evaluating predict_fall on fall-only test data...
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Classification Report for Fall Classes: ['fall_backward' 'fall_forward' 'fall_side_left' 'fall_side_right'
 'fall_slump']
                 precision    recall  f1-score   support

           bend       0.00      0.00      0.00         0
  fall_backward       1.00      0.96      0.98       590
   fall_forward       0.98      0.97      0.98       560
 fall_side_left       0.99      0.98      0.98       370
fall_side_right       0.99      0.98      0.98       470
     fall_slump       0.98      0.97      0.98       350
       lie_down       0.00      0.00      0.00         0
            sit       0.00      0.00      0.00         0
          stand       0.00      0.00      0.00         0
           walk       0.00      0.00      0.00         0

       accuracy                           0.97      2340
      macro avg       0.49      0.49      0.49      2340
   weighted avg       0.99      0.97      0.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
print("\nTesting 'fall detected' cases using samples from X_fall_test...")

# Test a few samples that are known to be fall-related
num_samples_to_test = 5

for i in range(num_samples_to_test):
    sample_fall = X_fall_test[i]
    true_label_fall = le.inverse_transform([y_fall_test_int[i]])[0]

    print(f"\n--- Testing sample {i+1} ---")
    print(f"True Label: {true_label_fall}")
    pred_label_fall = predict_fall(sample_fall)
    print(f"Predicted Label: {pred_label_fall}")


Testing 'fall detected' cases using samples from X_fall_test...

--- Testing sample 1 ---
True Label: fall_slump
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
FALL DETECTED
Predicted Label: fall_slump

--- Testing sample 2 ---
True Label: fall_backward
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
FALL DETECTED
Predicted Label: fall_backward

--- Testing sample 3 ---
True Label: fall_side_right
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
FALL DETECTED
Predicted Label: fall_side_right

--- Testing sample 4 ---
True Label: fall_side_left
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
FALL DETECTED
Predicted Label: fall_side_left

--- Testing sample 5 ---
True Label: fall_forward
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
FALL DETECTED
Predicted Label: fall_forward
